In [156]:
# generate data
import Distributions
using Random

Random.seed!(42)

n_samples = 10
dimensionality = 2
rθ = [-0.5, 0.2]
inputs = randn(n_samples, dimensionality)
σ(w, x) = 1/(1+exp(-w'x))
πs = [σ(rθ, inputs[i, :]) for i in 1:n_samples]
outputs = [rand(Distributions.Bernoulli(πs[i])) for i in 1:n_samples]

10-element Vector{Bool}:
 1
 1
 1
 1
 0
 1
 1
 0
 1
 1

In [233]:
# test nonlinear
using ForneyLab
import ForneyLab: unsafeMean, unsafeCov

graph = FactorGraph()

T = n_samples
x = Vector{Variable}(undef, T)
y = Vector{Variable}(undef, T)

# Hearing aid parameters
@RV θ  ~ GaussianMeanPrecision(placeholder(:m_θ, dims=(order,)), placeholder(:W_θ, dims=(order, order)))
f(w,x) = 1/(1+exp(-w'x))
for i in 1:T
    @eval $(Symbol("func$i"))(θ) = f(θ,inputs[$i, :])
    @RV x[i] ~ Nonlinear{Sampling}(θ, g=eval(Symbol("func$i")), in_variates=[Multivariate], out_variate=Univariate)
    @RV y[i] ~ Bernoulli(x[i])
    placeholder(y[i], :y, index=i)
end

In [234]:
algo = messagePassingAlgorithm(θ, free_energy=true)
src_code = algorithmSourceCode(algo, free_energy=true);

In [235]:
println(src_code);

begin

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 4))

messages[1] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=data[:m_θ]), Message(MatrixVariate, PointMass, m=data[:W_θ]))
messages[2] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass, m=data[:y][1]), nothing)
messages[3] = ruleSPNonlinearSIn1MN(func1, messages[2], nothing, variate=Multivariate)
messages[4] = ruleSPNonlinearSOutNM(func1, nothing, messages[1], variate=Univariate)

marginals[:x_1] = messages[4].dist * messages[2].dist
marginals[:θ] = messages[1].dist * messages[3].dist

return marginals

end

function freeEnergy(data::Dict, marginals::Dict)

F = 0.0

F += averageEnergy(Bernoulli, ProbabilityDistribution(Univariate, PointMass, m=data[:y][1]), marginals[:x_1])
F += averageEnergy(GaussianMeanPrecision, marginals[:θ], ProbabilityDistribution(Multivariate, PointMass, m=data[:m_θ]), ProbabilityDistribution(MatrixVariate, PointMass, m

In [236]:
# Load algorithm
eval(Meta.parse(src_code))

freeEnergy (generic function with 1 method)

In [237]:
data = Dict(:y => outputs, :m_θ => zeros(dimensionality), :W_θ => 0.1*diageye(dimensionality))
marginals = step!(data)

Dict{Any, Any} with 2 entries:
  :x_1 => SampleList(s=[0.80, 0.04, 0.59, 0.74, 1.00, 9.24e-03, 0.83, 0.97, 0.9…
  :θ   => 𝒩(m=[-0.71, 1.38], w=[[0.14, -0.07][-0.07, 0.23]])…

In [238]:
meθ = unsafeMean(marginals[:θ])

2-element Vector{Float64}:
 -0.7070884632433143
  1.3764429313586235

In [239]:
weθ = unsafeCov(marginals[:θ])

2×2 Matrix{Float64}:
 8.68621  2.55746
 2.55746  5.02156

In [291]:
println("training errors = $(sum(abs.([round(f(meθ, inputs[i, :])) - round(πs[i]) for i in 1:n_samples])))")

training errors = 1.0


In [375]:
# test nonlinear
using ForneyLab
using LinearAlgebra
import ForneyLab: unsafeMean, unsafeCov

graph = FactorGraph()

T = 2
x = Vector{Variable}(undef, T)
z = Vector{Variable}(undef, T)
y = Vector{Variable}(undef, T)

# Hearing aid parameters
@RV θ  ~ GaussianMeanPrecision(placeholder(:m_θ, dims=(dimensionality,)), placeholder(:W_θ, dims=(dimensionality, dimensionality)))
f(w, x) = 1/(1+exp(-w'x))
for i in 1:T
    @RV z[i] ~ GaussianMeanPrecision(inputs[i, :], 1e4*diageye(dimensionality))
    @RV x[i] ~ Nonlinear{Sampling}(θ, z[i], g=f, in_variates=[Multivariate, Multivariate], out_variate=Univariate)
    @RV y[i] ~ Bernoulli(x[i])
    placeholder(y[i], :y, index=i)
end

In [376]:
draw()

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.47.2 (20210527.0053)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 4272520496268082245 -->
 
 4272520496268082245 
 
 placeholder_y_1 
 
<!-- 15006462186345505204 -->
 
 15006462186345505204 
 
 Ber 
 bernoulli_1 
 
<!-- 4272520496268082245--15006462186345505204 -->
 
 4272520496268082245--15006462186345505204 
 
 y_1 
 1 out 
 1 out 
 
<!-- 10214283830915002291 -->
 
 10214283830915002291 
 
 placeholder_y_2 
 
<!-- 8224144393233565915 -->
 
 8224144393233565915 
 
 Ber 
 bernoulli_2 
 
<!-- 10214283830915002291--8224144393233565915 -->
 
 10214283830915002291--8224144393233565915 
 
 y_2 
 1 out 
 1 out 
 
<!-- 1183587289073611544 -->
 
 1183587289073611544 
 
 clamp_4 
 
<!-- 13558487332618118094 -->
 
 13558487332618118094 
 
 g{Sampling} 
 nonlinear_1 
 
<!-- 15006462186345505204--13558487332618118094 -->
 
 15006462186345505204--13558487332618118094 
 
 x_1 
 1 out 
 2 p 
 
<!-- 9018848755142160459 -->
 
 9018848755142160459 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 143706769862255191 -->
 
 143706769862255191 
 
 placeholder_W_θ 
 
<!-- 9018848755142160459--143706769862255191 -->
 
 9018848755142160459--143706769862255191 
 
 W_θ 
 1 out 
 3 w 
 
<!-- 15806283522943940197 -->
 
 15806283522943940197 
 
 placeholder_m_θ 
 
<!-- 9018848755142160459--15806283522943940197 -->
 
 9018848755142160459--15806283522943940197 
 
 m_θ 
 1 out 
 2 m 
 
<!-- 4042784542115340737 -->
 
 4042784542115340737 
 
 g{Sampling} 
 nonlinear_2 
 
<!-- 815416855342792530 -->
 
 815416855342792530 
 
 𝒩 
 gaussianmeanprecision_3 
 
<!-- 4042784542115340737--815416855342792530 -->
 
 4042784542115340737--815416855342792530 
 
 z_2 
 1 out 
 3 in2 
 
<!-- 8224144393233565915--4042784542115340737 -->
 
 8224144393233565915--4042784542115340737 
 
 x_2 
 1 out 
 2 p 
 
<!-- 6650370946060692794 -->
 
 6650370946060692794 
 
 = 
 equ_θ_1 
 
<!-- 6650370946060692794--9018848755142160459 -->
 
 6650370946060692794--9018848755142160459 
 
 θ 
 1 out 
 1 1 
 
<!-- 6650370946060692794--4042784542115340737 -->
 
 6650370946060692794--4042784542115340737 
 
 θ 
 2 in1 
 3 3 
 
<!-- 6650370946060692794--13558487332618118094 -->
 
 6650370946060692794--13558487332618118094 
 
 θ 
 2 in1 
 2 2 
 
<!-- 15353340341171232116 -->
 
 15353340341171232116 
 
 clamp_2 
 
<!-- 815416855342792530--1183587289073611544 -->
 
 815416855342792530--1183587289073611544 
 
 clamp_4 
 1 out 
 3 w 
 
<!-- 1903691017461356593 -->
 
 1903691017461356593 
 
 clamp_3 
 
<!-- 815416855342792530--1903691017461356593 -->
 
 815416855342792530--1903691017461356593 
 
 clamp_3 
 1 out 
 2 m 
 
<!-- 10661251336546118771 -->
 
 10661251336546118771 
 
 𝒩 
 gaussianmeanprecision_2 
 
<!-- 13558487332618118094--10661251336546118771 -->
 
 13558487332618118094--10661251336546118771 
 
 z_1 
 1 out 
 3 in2 
 
<!-- 10661251336546118771--15353340341171232116 -->
 
 10661251336546118771--15353340341171232116 
 
 clamp_2 
 1 out 
 3 w 
 
<!-- 9045958292410383509 -->
 
 9045958292410383509 
 
 clamp_1 
 
<!-- 10661251336546118771--9045958292410383509 -->
 
 10661251336546118771--9045958292410383509 
 
 clamp_1 
 1 out 
 2 m

In [377]:
# Define posterior factorization
pfz = PosteriorFactorization()

PosteriorFactorization(FactorGraph(Dict{Symbol, FactorNode}(:nonlinear_1 => Nonlinear{Sampling} with id nonlinear_1
, :equ_θ_1 => Equality with id equ_θ_1
, :clamp_3 => Clamp{Multivariate} with id clamp_3
, :placeholder_W_θ => Clamp{MatrixVariate} with id placeholder_W_θ
, :clamp_1 => Clamp{Multivariate} with id clamp_1
, :bernoulli_1 => Bernoulli with id bernoulli_1
, :gaussianmeanprecision_1 => GaussianMeanPrecision with id gaussianmeanprecision_1
, :gaussianmeanprecision_2 => GaussianMeanPrecision with id gaussianmeanprecision_2
, :clamp_2 => Clamp{MatrixVariate} with id clamp_2
, :placeholder_y_2 => Clamp{Univariate} with id placeholder_y_2
…), Edges:
Edge belonging to variable m_θ: ( placeholder_m_θ.i[out] )----( gaussianmeanprecision_1.i[m] ).
Edge belonging to variable W_θ: ( placeholder_W_θ.i[out] )----( gaussianmeanprecision_1.i[w] ).
Edge belonging to variable θ: ( gaussianmeanprecision_1.i[out] )----( equ_θ_1.i[1] ).
Edge belonging to variable clamp_1: ( clamp_1.i[out] )----

In [378]:
# Compile algorithm
algo = messagePassingAlgorithm(free_energy=true)

# Generate source code
src_code = algorithmSourceCode(algo, free_energy=true);

In [379]:
println(src_code)

begin

function init()

messages = Array{Message}(undef, 14)

messages[1] = Message(vague(GaussianMeanPrecision))
messages[2] = Message(vague(GaussianMeanPrecision))
messages[6] = Message(vague(GaussianWeightedMeanPrecision))
messages[9] = Message(vague(GaussianWeightedMeanPrecision))

return messages

end

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 14))

messages[1] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=[-0.444383357109696, 0.18702790710363]), Message(MatrixVariate, PointMass, m=Diagonal([10000.0, 10000.0])))
messages[2] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=[-0.5560268761463861, 1.0823812056084292]), Message(MatrixVariate, PointMass, m=Diagonal([10000.0, 10000.0])))
messages[3] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=data[:m_θ]), Message(MatrixVariate, PointMass, m=data[:W_θ]))
messages[4] = ruleSPBernoulli

In [380]:
# Load algorithm
eval(Meta.parse(src_code))

freeEnergy (generic function with 1 method)

In [381]:
ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=[0.15614346264074028, -1.590579974922555]), Message(MatrixVariate, PointMass, m=Diagonal([10000.0, 10000.0])))

Message: 𝒩(m=[0.16, -1.59], w=diag[1.00e+04, 1.00e+04])


In [384]:
function init()
    messages = Array{Message}(undef, 14)
    for i in 1:length(messages)
        messages[i] = Message(vague(GaussianMeanPrecision, 2))
    end
    messages
end

init (generic function with 1 method)

In [385]:
data = Dict(:y => outputs, :m_θ => zeros(dimensionality), :W_θ => 0.1*diageye(dimensionality))
messages = init()
marginals = Dict()
step!(data, marginals, messages)

LoadError: MethodError: no method matching ruleSPNonlinearSOutNGX(::typeof(f), ::Nothing, ::Message{GaussianWeightedMeanPrecision, Multivariate}, ::Message{GaussianMeanPrecision, Multivariate}; variate=Univariate)
[0mClosest candidates are:
[0m  ruleSPNonlinearSOutNGX(::Function, ::Nothing, ::Message{var"#s158", V} where var"#s158"<:Gaussian...; n_samples) where V<:ForneyLab.VariateType at /Users/apodusenko/.julia/dev/ForneyLab/src/engines/julia/update_rules/nonlinear_sampling.jl:73[91m got unsupported keyword argument "variate"[39m
[0m  ruleSPNonlinearSOutNGX(::Function, ::Nothing, ::Message{var"#s148", var"#s147"} where {var"#s148"<:Gaussian, var"#s147"<:ForneyLab.VariateType}...; n_samples, variate) at /Users/apodusenko/.julia/dev/ForneyLab/src/engines/julia/update_rules/nonlinear_sampling.jl:65

In [383]:
messages

14-element Vector{Message}:
    Message: 𝒩(m=[-0.44, 0.19], w=diag[1.00e+04, 1.00e+04])

    Message: 𝒩(m=[-0.56, 1.08], w=diag[1.00e+04, 1.00e+04])

    Message: 𝒩(m=[0.00, 0.00], w=diag[0.10, 0.10])

    Message: Beta(a=2, b=1)

 #undef
    Message: 𝒩(xi=0.00, w=1.00e-12)

 #undef
 #undef
    Message: 𝒩(xi=0.00, w=1.00e-12)

 #undef
 #undef
 #undef
 #undef
 #undef

In [391]:
messages[1] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=[-0.444383357109696, 0.18702790710363]), Message(MatrixVariate, PointMass, m=Diagonal([10000.0, 10000.0])))
messages[2] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=[-0.5560268761463861, 1.0823812056084292]), Message(MatrixVariate, PointMass, m=Diagonal([10000.0, 10000.0])))
messages[3] = ruleSPGaussianMeanPrecisionOutNPP(nothing, Message(Multivariate, PointMass, m=data[:m_θ]), Message(MatrixVariate, PointMass, m=data[:W_θ]))
messages[4] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass, m=data[:y][1]), nothing)
messages[5] = ruleSPNonlinearSInGX(f, 1, messages[4], messages[9], messages[2], variate=Multivariate)
messages[6] = ruleSPEqualityGaussian(messages[3], messages[5], nothing)
messages[7] = ruleSPBernoulliIn1PN(Message(Univariate, PointMass, m=data[:y][2]), nothing)
messages[8] = ruleSPNonlinearSInGX(f, 1, messages[7], messages[6], messages[1], variate=Multivariate)
messages[9] = ruleSPEqualityGaussian(messages[3], nothing, messages[8])
messages[10] = ruleSPEqualityGaussian(nothing, messages[5], messages[8])
messages[11] = ruleSPNonlinearSInGX(f, 2, messages[7], messages[6], messages[1], variate=Multivariate)
messages[12] = ruleSPNonlinearSInGX(f, 2, messages[4], messages[9], messages[2], variate=Multivariate)
messages[13] = ruleSPNonlinearSOutNGX(f, nothing, messages[6], messages[1], variate=Univariate)

LoadError: MethodError: no method matching ruleSPNonlinearSOutNGX(::typeof(f), ::Nothing, ::Message{GaussianWeightedMeanPrecision, Multivariate}, ::Message{GaussianMeanPrecision, Multivariate}; variate=Univariate)
[0mClosest candidates are:
[0m  ruleSPNonlinearSOutNGX(::Function, ::Nothing, ::Message{var"#s158", V} where var"#s158"<:Gaussian...; n_samples) where V<:ForneyLab.VariateType at /Users/apodusenko/.julia/dev/ForneyLab/src/engines/julia/update_rules/nonlinear_sampling.jl:73[91m got unsupported keyword argument "variate"[39m
[0m  ruleSPNonlinearSOutNGX(::Function, ::Nothing, ::Message{var"#s148", var"#s147"} where {var"#s148"<:Gaussian, var"#s147"<:ForneyLab.VariateType}...; n_samples, variate) at /Users/apodusenko/.julia/dev/ForneyLab/src/engines/julia/update_rules/nonlinear_sampling.jl:65

In [365]:
?ruleSPNonlinearSOutNGX

search: ruleSPNonlinearSOutNGX ruleSPNonlinearSOutNM ruleSPNonlinearSOutNFactorX



No documentation found.

`ForneyLab.ruleSPNonlinearSOutNGX` is a `Function`.

```
# 2 methods for generic function "ruleSPNonlinearSOutNGX":
[1] ruleSPNonlinearSOutNGX(g::Function, msg_out::Nothing, msgs_in::Message{var"#s158", V} where var"#s158"<:Gaussian...; n_samples) where V<:ForneyLab.VariateType in ForneyLab at /Users/apodusenko/.julia/dev/ForneyLab/src/engines/julia/update_rules/nonlinear_sampling.jl:73
[2] ruleSPNonlinearSOutNGX(g::Function, msg_out::Nothing, msgs_in::Message{var"#s148", var"#s147"} where {var"#s148"<:Gaussian, var"#s147"<:ForneyLab.VariateType}...; n_samples, variate) in ForneyLab at /Users/apodusenko/.julia/dev/ForneyLab/src/engines/julia/update_rules/nonlinear_sampling.jl:65
```
